<a href="https://colab.research.google.com/github/vishnu2005/VAYNOMICS/blob/main/uber_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn joblib


In [ ]:
from google.colab import files

# This will prompt you to upload your file manually
uploaded = files.upload()


Saving uber_fare_dataset_quality.csv to uber_fare_dataset_quality.csv


In [ ]:
import pandas as pd

# Automatically get the uploaded filename
filename = list(uploaded.keys())[0]

# Load the dataset
df = pd.read_csv(filename)
df.head()


,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,hour_of_day,day_of_week,is_peak,ride_type,city,fare_amount
0,13.012362,80.290143,13.119598,80.219732,6,1,0,uber_go,chennai,252.15
1,13.112422,80.104117,13.190973,80.266489,5,1,0,uber_go,chennai,340.41
2,12.954547,80.136681,12.991273,80.204951,11,0,0,uber_go,chennai,178.63
3,12.941848,80.158429,13.009909,80.191214,14,5,0,uber_go,chennai,152.16
4,12.959902,80.202847,13.077724,80.109290,18,6,1,uber_go,chennai,362.93


In [ ]:
# Check for nulls
print("Missing values:\n", df.isnull().sum())

# Map ride type and city
ride_type_map = {'uber_go': 0, 'uber_auto': 1, 'uber_xl': 2}
city_map = {'chennai': 0, 'bengaluru': 1, 'mumbai': 2, 'delhi': 3}

df['ride_type_encoded'] = df['ride_type'].str.lower().map(ride_type_map)
df['city_encoded'] = df['city'].str.lower().map(city_map)


Missing values:
 pickup_latitude      0
pickup_longitude     0
dropoff_latitude     0
dropoff_longitude    0
hour_of_day          0
day_of_week          0
is_peak              0
ride_type            0
city                 0
fare_amount          0
dtype: int64


In [ ]:
features = [
    'pickup_latitude', 'pickup_longitude',
    'dropoff_latitude', 'dropoff_longitude',
    'hour_of_day', 'day_of_week', 'is_peak',
    'ride_type_encoded', 'city_encoded'
]
target = 'fare_amount'

X = df[features]
y = df[target]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=42)
model.fit(X_train, y_train)

# Predict & Evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"✅ R² Score: {r2:.4f}")
print(f"📉 RMSE: ₹{rmse:.2f}")


✅ R² Score: 0.8999
📉 RMSE: ₹55.53


In [ ]:
import joblib

# Save model
joblib.dump(model, 'multitype_uber_model.pkl')
print("🎉 Model saved as 'multitype_uber_model.pkl'")

# Download it to your system
from google.colab import files
files.download('multitype_uber_model.pkl')


🎉 Model saved as 'multitype_uber_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Load the CSV
df = pd.read_csv("final_multitype_data.csv")

# ✅ Ensure only 8 columns (EXCLUDE 'city_encoded')
features = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
    'hour', 'day_of_week', 'is_peak', 'ride_type_encoded'
]

X = df[features]
y = df["fare"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print(f"✅ R² Score: {r2:.4f}")
print(f"📉 RMSE: ₹{rmse:.2f}")

# Save model
joblib.dump(model, "multitype_uber_model.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'final_multitype_data.csv'

# Training with different dataset to get more R^2 and RMSE

In [ ]:
# ✅ STEP 1: Install dependencies
!pip install pandas scikit-learn joblib

# ✅ STEP 2: Imports
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import random

# ✅ STEP 3: Define helper functions
def generate_coordinates(city_center, radius_km):
    lat_center, lng_center = city_center
    radius_deg = radius_km / 111  # Approximate degrees in latitude

    lat_offset = np.random.uniform(-radius_deg, radius_deg)
    lng_offset = np.random.uniform(-radius_deg, radius_deg)

    return lat_center + lat_offset, lng_center + lng_offset

def calculate_fare(distance_km, ride_type, is_peak):
    base_fare = {
        0: 45,  # Uber Go
        1: 30,  # Uber Auto
        2: 85   # Uber XL
    }[ride_type]

    per_km = {
        0: 12,
        1: 9,
        2: 20
    }[ride_type]

    surge = 1.3 if is_peak else 1.0

    fare = (base_fare + distance_km * per_km) * surge
    return round(fare + random.uniform(-5, 5), 2)  # Add slight noise

def haversine(lat1, lon1, lat2, lon2):
    from math import radians, cos, sin, asin, sqrt
    R = 6371  # Earth radius in km

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return R * 2 * asin(sqrt(a))

# ✅ STEP 4: Define cities (optional: used only for generating data, not in model)
cities = {
    "Chennai": (13.0827, 80.2707),
    "Bengaluru": (12.9716, 77.5946),
    "Delhi": (28.6139, 77.2090),
    "Mumbai": (19.0760, 72.8777)
}

# ✅ STEP 5: Generate synthetic dataset
rows = []
for _ in range(10000):
    city_name = random.choice(list(cities.keys()))
    center = cities[city_name]

    pickup_lat, pickup_lng = generate_coordinates(center, radius_km=10)
    dropoff_lat, dropoff_lng = generate_coordinates(center, radius_km=10)

    hour = random.randint(0, 23)
    day_of_week = random.randint(0, 6)
    is_peak = 1 if 7 <= hour <= 10 or 17 <= hour <= 20 else 0
    ride_type = random.choice([0, 1, 2])

    distance_km = haversine(pickup_lat, pickup_lng, dropoff_lat, dropoff_lng)
    fare = calculate_fare(distance_km, ride_type, is_peak)

    rows.append([
        pickup_lat, pickup_lng, dropoff_lat, dropoff_lng,
        hour, day_of_week, is_peak, ride_type, fare
    ])

df = pd.DataFrame(rows, columns=[
    "pickup_latitude", "pickup_longitude",
    "dropoff_latitude", "dropoff_longitude",
    "hour", "day_of_week", "is_peak", "ride_type",
    "fare_amount"
])

# ✅ STEP 6: Save synthetic dataset (optional)
df.to_csv("final_multitype_data.csv", index=False)

# ✅ STEP 7: Prepare data
X = df.drop("fare_amount", axis=1)
y = df["fare_amount"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ STEP 8: Train Random Forest model
model = RandomForestRegressor(n_estimators=150, random_state=42)
model.fit(X_train, y_train)

# ✅ STEP 9: Evaluate model
y_pred = model.predict(X_test)
print("🔍 R² Score:", r2_score(y_test, y_pred))
print("📉 RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("📊 MAE:", mean_absolute_error(y_test, y_pred))

# ✅ STEP 10: Save model
joblib.dump(model, "multitype_uber_model.pkl")
print("✅ Model saved as multitype_uber_model.pkl")


🔍 R² Score: 0.8912225882505241
📉 RMSE: 37.895311261558554
📊 MAE: 28.3201357
✅ Model saved as multitype_uber_model.pkl


In [ ]:
from google.colab import files

# Download the model
files.download('multitype_uber_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>